# Benchmarking skforecast Direct Forecasters

This notebook benchmarks the performance (velocity) of the `skforecast` in its different versions and keeps track of the results.

**Notes**

+ In version `0.15.0` the binning of residuals was introduced in multi-series forecasters. This explains the increase in the time taken to fit the model.

In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

/home/joaquin/Documents/GitHub/skforecast


In [2]:
# !pip install --upgrade numpy < 2.0 && \
# pip install --upgrade pandas && \
# pip install --upgrade scikit-learn

# !pip install skforecast==0.15.1
# !pip install skforecast==0.14.0
# !pip install skforecast==0.13.0

In [ ]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
import sklearn
import joblib
from benchmarking import (
    plot_benchmark_results,
    run_benchmark_ForecasterDirectMultiVariate,
    run_benchmark_ForecasterDirect
)
import skforecast
import platform
import psutil
import plotly.express as px
import warnings

if skforecast.__version__ >= '0.17.0':
    warnings.simplefilter('ignore', category=skforecast.exceptions.InputTypeWarning)

In [4]:
print(f"Python version: {platform.python_version()}")
print(f"skforecast version: {skforecast.__version__}")
print(f"scikit-learn version: {sklearn.__version__}")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")
print(f"Computer network name: {platform.node()}")
print(f"Processor type: {platform.processor()}")
print(f"Platform type: {platform.platform()}")
print(f"Operating system: {platform.system()}")
print(f"Operating system release: {platform.release()}")
print(f"Operating system version: {platform.version()}")
print(f"Number of physical cores: {psutil.cpu_count(logical=False)}")
print(f"Number of logical cores: {psutil.cpu_count(logical=True)}")

Python version: 3.12.9
skforecast version: 0.16.0
scikit-learn version: 1.6.1
pandas version: 2.2.3
numpy version: 2.2.5
Computer network name: joaquin-HP-ProBook-440-G6
Processor type: x86_64
Platform type: Linux-6.11.0-24-generic-x86_64-with-glibc2.39
Operating system: Linux
Operating system release: 6.11.0-24-generic
Operating system version: #24~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Tue Mar 25 20:14:34 UTC 2
Number of physical cores: 4
Number of logical cores: 8


In [5]:
import warnings
warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message="'force_all_finite' was renamed to 'ensure_all_finite'"
)

# ForecasterDirectMultivariate

In [6]:
# Mock data for benchmarking
# ==========================================================
n_series = 13
len_series = 1000
rng = np.random.default_rng(321)
series = pd.DataFrame(
    data = rng.normal(
        loc = 10,
        scale = 3,
        size = (len_series, n_series)
    ),
    columns = [f'series_{i}' for i in range(n_series)],
    index = pd.date_range(
        start = '2020-01-01',
        periods = len_series,
        freq = 'h'
    )
)

exog = pd.DataFrame(index=series.index)
exog['day_of_week'] = exog.index.dayofweek
exog['week_of_year'] = exog.index.isocalendar().week.astype(int)
exog['month'] = exog.index.month

exog_prediction = pd.DataFrame(
        index=pd.date_range(
                start=series.index.max() + pd.Timedelta(hours=1),
                periods=10,
                freq='h'
            ),
        )
exog_prediction['day_of_week'] = exog_prediction.index.dayofweek
exog_prediction['week_of_year'] = exog_prediction.index.isocalendar().week.astype(int)
exog_prediction['month'] = exog_prediction.index.month

In [7]:
run_benchmark_ForecasterDirectMultiVariate(series=series, exog=exog, exog_prediction=exog_prediction)

Benchmarking function: ForecasterDirectMultiVariate__create_train_X_y
Benchmarking function: ForecasterDirectMultiVariate__create_train_X_y_no_exog
Benchmarking function: ForecasterDirectMultiVariate_fit
Benchmarking function: ForecasterDirectMultiVariate_fit_series_no_exog
Benchmarking function: ForecasterDirectMultiVariate_predict
Benchmarking function: ForecasterDirectMultiVariate_predict_interval_conformal
Benchmarking function: ForecasterDirectMultiVariate__create_predict_inputs
Benchmarking function: ForecasterDirectMultiVariate__check_predict_inputs
Benchmarking function: ForecasterDirectMultiVariate_backtesting
Benchmarking function: ForecasterDirectMultiVariate_backtesting_no_exog
Benchmarking function: ForecasterDirectMultiVariate_backtesting_conformal


In [8]:
# Plot results
# ==============================================================================
display_df = True
selected_date = None
# 'Linux-6.11.0-24-generic-x86_64-with-glibc2.39'
# 'Windows-10-10.0.19045-SP0'
selected_platform = None

results_benchmark_all = joblib.load("./benchmark.joblib")
results_benchmark = results_benchmark_all.query("forecaster_name in ['ForecasterDirectMultiVariate', 'ForecasterDirectMultiVariate']")
results_benchmark = results_benchmark.query("regressor_name == 'LGBMRegressor'")
for function_name in results_benchmark['function_name'].unique():
    df = results_benchmark.query(f"function_name == '{function_name}'")
    if selected_date:
        df = df[df['datetime'].dt.date == pd.to_datetime(selected_date).date()]
    if selected_platform:
        df = df[df['platform'] == selected_platform]
    if display_df:
        display(df.tail(3))
    plot_benchmark_results(df.copy(), function_name, add_median=True, add_mean=True)

,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
448,ForecasterDirectMultiVariate,LGBMRegressor,ForecasterDirectMultiVariate__create_train_X_y,376309f7138e749dc0a50d758cde50bb,0.031474,0.004858,2025-04-28 10:12:27.980945,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64
466,ForecasterDirectMultiVariate,LGBMRegressor,ForecasterDirectMultiVariate__create_train_X_y,376309f7138e749dc0a50d758cde50bb,0.046757,0.013822,2025-04-28 10:19:57.916943,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64
484,ForecasterDirectMultiVariate,LGBMRegressor,ForecasterDirectMultiVariate__create_train_X_y,376309f7138e749dc0a50d758cde50bb,0.016028,0.004921,2025-04-28 10:22:48.235104,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
449,ForecasterDirectMultiVariate,LGBMRegressor,ForecasterDirectMultiVariate__create_train_X_y...,b372eea4a4311f51ded53db33ae6294f,0.024050,0.001419,2025-04-28 10:12:28.227009,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64
467,ForecasterDirectMultiVariate,LGBMRegressor,ForecasterDirectMultiVariate__create_train_X_y...,b372eea4a4311f51ded53db33ae6294f,0.028180,0.002302,2025-04-28 10:19:58.206016,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64
485,ForecasterDirectMultiVariate,LGBMRegressor,ForecasterDirectMultiVariate__create_train_X_y...,b372eea4a4311f51ded53db33ae6294f,0.009469,0.000857,2025-04-28 10:22:48.335669,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
450,ForecasterDirectMultiVariate,LGBMRegressor,ForecasterDirectMultiVariate_fit,47471f487e0e7d61036a1954e2805bab,3.385594,0.068871,2025-04-28 10:12:45.160132,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64
468,ForecasterDirectMultiVariate,LGBMRegressor,ForecasterDirectMultiVariate_fit,47471f487e0e7d61036a1954e2805bab,3.875952,0.395109,2025-04-28 10:20:17.592242,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64
486,ForecasterDirectMultiVariate,LGBMRegressor,ForecasterDirectMultiVariate_fit,47471f487e0e7d61036a1954e2805bab,4.366452,0.537333,2025-04-28 10:23:10.173709,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
451,ForecasterDirectMultiVariate,LGBMRegressor,ForecasterDirectMultiVariate_fit_series_no_exog,9b50bdc77fb5ebc83bef7062a929329d,3.808646,0.223725,2025-04-28 10:13:04.208050,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64
469,ForecasterDirectMultiVariate,LGBMRegressor,ForecasterDirectMultiVariate_fit_series_no_exog,9b50bdc77fb5ebc83bef7062a929329d,4.699641,0.951723,2025-04-28 10:20:41.095133,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64
487,ForecasterDirectMultiVariate,LGBMRegressor,ForecasterDirectMultiVariate_fit_series_no_exog,9b50bdc77fb5ebc83bef7062a929329d,5.164501,0.419578,2025-04-28 10:23:36.007210,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
452,ForecasterDirectMultiVariate,LGBMRegressor,ForecasterDirectMultiVariate_predict,8b127a277991b7d6f02e1ed2b721c8c2,0.012432,0.000718,2025-04-28 10:13:08.370107,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64
470,ForecasterDirectMultiVariate,LGBMRegressor,ForecasterDirectMultiVariate_predict,8b127a277991b7d6f02e1ed2b721c8c2,0.013093,0.001566,2025-04-28 10:20:45.574496,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64
488,ForecasterDirectMultiVariate,LGBMRegressor,ForecasterDirectMultiVariate_predict,8b127a277991b7d6f02e1ed2b721c8c2,0.011929,0.000600,2025-04-28 10:23:40.219596,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64


...

,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
454,ForecasterDirectMultiVariate,LGBMRegressor,ForecasterDirectMultiVariate__create_predict_i...,23899bed19ca242e9e6f882ee640d602,0.006411,0.000354,2025-04-28 10:13:08.447914,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64
472,ForecasterDirectMultiVariate,LGBMRegressor,ForecasterDirectMultiVariate__create_predict_i...,23899bed19ca242e9e6f882ee640d602,0.006728,0.000510,2025-04-28 10:20:45.655766,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64
490,ForecasterDirectMultiVariate,LGBMRegressor,ForecasterDirectMultiVariate__create_predict_i...,23899bed19ca242e9e6f882ee640d602,0.005378,0.000247,2025-04-28 10:23:40.412284,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
455,ForecasterDirectMultiVariate,LGBMRegressor,ForecasterDirectMultiVariate__check_predict_in...,07dd51bd3a759c550b57be0e6d738b8c,0.000918,0.000187,2025-04-28 10:13:08.462646,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64
473,ForecasterDirectMultiVariate,LGBMRegressor,ForecasterDirectMultiVariate__check_predict_in...,07dd51bd3a759c550b57be0e6d738b8c,0.001040,0.000170,2025-04-28 10:20:45.672521,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64
491,ForecasterDirectMultiVariate,LGBMRegressor,ForecasterDirectMultiVariate__check_predict_in...,07dd51bd3a759c550b57be0e6d738b8c,0.000291,0.000045,2025-04-28 10:23:40.420277,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
456,ForecasterDirectMultiVariate,LGBMRegressor,ForecasterDirectMultiVariate_backtesting,e349f4f06ff17af15c1de99814e50f92,4.112763,0.048103,2025-04-28 10:13:29.031576,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64
474,ForecasterDirectMultiVariate,LGBMRegressor,ForecasterDirectMultiVariate_backtesting,e349f4f06ff17af15c1de99814e50f92,5.480873,0.683325,2025-04-28 10:21:13.083334,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64
492,ForecasterDirectMultiVariate,LGBMRegressor,ForecasterDirectMultiVariate_backtesting,e349f4f06ff17af15c1de99814e50f92,4.415023,0.201772,2025-04-28 10:24:02.501343,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
457,ForecasterDirectMultiVariate,LGBMRegressor,ForecasterDirectMultiVariate_backtesting_no_exog,7858762c489123dbea6f9684ec906563,3.974546,0.053036,2025-04-28 10:13:48.910101,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64
475,ForecasterDirectMultiVariate,LGBMRegressor,ForecasterDirectMultiVariate_backtesting_no_exog,7858762c489123dbea6f9684ec906563,4.882481,0.441004,2025-04-28 10:21:37.504861,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64
493,ForecasterDirectMultiVariate,LGBMRegressor,ForecasterDirectMultiVariate_backtesting_no_exog,7858762c489123dbea6f9684ec906563,4.384573,0.176566,2025-04-28 10:24:24.430435,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
458,ForecasterDirectMultiVariate,LGBMRegressor,ForecasterDirectMultiVariate_backtesting_confo...,4351b2618048e0b6126eabb9a16ae17a,NaN,NaN,2025-04-28 10:13:48.916006,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64
476,ForecasterDirectMultiVariate,LGBMRegressor,ForecasterDirectMultiVariate_backtesting_confo...,4351b2618048e0b6126eabb9a16ae17a,NaN,NaN,2025-04-28 10:21:37.511647,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64
494,ForecasterDirectMultiVariate,LGBMRegressor,ForecasterDirectMultiVariate_backtesting_confo...,4351b2618048e0b6126eabb9a16ae17a,4.508099,0.166571,2025-04-28 10:24:46.978274,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64


# ForecasterDirect

In [9]:
# Mock data for benchmarking
# ==========================================================
len_series = 2000
rng = np.random.default_rng(321)
y = pd.Series(
        data = rng.normal(loc=20, scale=5, size=len_series),
        index=pd.date_range(
            start='2010-01-01',
            periods=len_series,
            freq='h'
        ),
        name='y'
    )
rng = np.random.default_rng(321)
exog = pd.DataFrame(index=y.index)
exog['day_of_week'] = exog.index.dayofweek
exog['week_of_year'] = exog.index.isocalendar().week.astype(int)
exog['month'] = exog.index.month
exog_prediction = pd.DataFrame(
                    index=pd.date_range(
                        start=exog.index.max() + pd.Timedelta(hours=1),
                        periods=100,
                        freq='h'
                    )
                 ) 
exog_prediction['day_of_week'] = exog_prediction.index.dayofweek
exog_prediction['week_of_year'] = exog_prediction.index.isocalendar().week.astype(int)
exog_prediction['month'] = exog_prediction.index.month

In [10]:
run_benchmark_ForecasterDirect(y=y, exog=exog, exog_prediction=exog_prediction)

Benchmarking function: ForecasterDirect__create_train_X_y
Benchmarking function: ForecasterDirect_fit
Benchmarking function: ForecasterDirect_predict
Benchmarking function: ForecasterDirect_predict_interval_conformal
Benchmarking function: ForecasterDirect__create_predict_inputs
Benchmarking function: ForecasterDirect_backtesting
Benchmarking function: ForecasterDirect_backtesting_conformal


In [11]:
# Plot results
# ==============================================================================
display_df = False
selected_date = None
# 'Linux-6.11.0-24-generic-x86_64-with-glibc2.39'
# 'Windows-10-10.0.19045-SP0'
selected_platform = None

results_benchmark_all = joblib.load("./benchmark.joblib")
results_benchmark = results_benchmark_all.query("forecaster_name in ['ForecasterDirect', 'ForecasterAutoregDirect']")
results_benchmark = results_benchmark.query("regressor_name == 'LGBMRegressor'")
for function_name in results_benchmark['function_name'].unique():
    df = results_benchmark.query(f"function_name == '{function_name}'")
    if selected_date:
        df = df[df['datetime'].dt.date == pd.to_datetime(selected_date).date()]
    if selected_platform:
        df = df[df['platform'] == selected_platform]
    if display_df:
        display(df)
    plot_benchmark_results(df.copy(), function_name, add_median=True, add_mean=True)

# Summary of historical results

In [ ]:
selected_date = None
# 'Linux-6.11.0-24-generic-x86_64-with-glibc2.39'
# 'Windows-10-10.0.19045-SP0'
selected_platform = None

results_benchmark_all = joblib.load("./benchmark.joblib")
results_benchmark_all['function_name'] = results_benchmark_all['function_name'].str.split('_', n=1).str[1]
if selected_date:
    results_benchmark_all = results_benchmark_all.query("datetime.dt.date == @selected_date")
if selected_platform:
    results_benchmark_all = results_benchmark_all.query("platform == @selected_platform")
results_benchmark_all = results_benchmark_all.groupby(['forecaster_name', 'skforecast_version', 'function_name'])['run_time_avg'].agg('median').reset_index()
results_benchmark_all = results_benchmark_all.sort_values(by=['function_name'])

fig = px.bar(
    results_benchmark_all.query("forecaster_name in ['ForecasterDirectMultiVariate', 'ForecasterAutoregMultiVariate']"),
    x='function_name',
    y='run_time_avg',
    color='skforecast_version',
    barmode='group',
    title='MultiVariate Forecasters - Median Run Time by Function and skforecast Version',
    labels={'run_time_avg': 'Median Run Time (s)', 'function_name': 'Function'},
    category_orders={'skforecast_version': sorted(results_benchmark_all['skforecast_version'].unique())}
)
fig.update_layout(xaxis_tickangle=-45, height=600)
fig.show()

fig = px.bar(
    results_benchmark_all.query("forecaster_name in ['ForecasterDirect', 'ForecasterAutoregDirect']"),
    x='function_name',
    y='run_time_avg',
    color='skforecast_version',
    barmode='group',
    title='Single-Series Forecasters - Median Run Time by Function and skforecast Version',
    labels={'run_time_avg': 'Median Run Time (s)', 'function_name': 'Function'},
    category_orders={'skforecast_version': sorted(results_benchmark_all['skforecast_version'].unique())}
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()
